# Urban Network Analysis
## Trip Assignment – Gradient Projection Algorithm

## Introduction

Traffic assignment algorithms can be broadly divided in link based and path based algorithms. Frank Wolfe, the most traditional algorithm is classified as link-based as its solution is the flow in each link; there is no information about the alternative routes for a given origin destination. 

On the other hand, path-based algorithms provide the solution in path flows and the link flows are readily obtained by the path-link incidence matrix. ALthough the method varies, one advantage of path-based is a higher flexibility of the solution as it is possible to change the proportion of path flows for a given origin destination pair which does not happen in, for example, Frank-Wolfe algorithm.

Here the gradient projection algorithm is briefly presented.

## Prerequisites

### 1. Packages and Modules Imported

> *** import networkx as nx ***

<ul><ul>: Search shortest path.
</ul></ul>

> *** import math 
      from scipy.misc import derivative ***

<ul><ul>: power, derivatives for GP
</ul></ul>

> *** import visualize_graph as v ***

<ul><ul>: Export networkX graph to ArcGIS shape format
</ul></ul>


In [1]:
import networkx as nx
import math
#import random
import matplotlib.pyplot as plt
import os
import sys
import pytrans.UrbanNetworkAnalysis.visualize_graph as v
from scipy.misc import derivative
import pytrans.UrbanNetworkAnalysis.TransportationNetworks as loadProblem

### 2. Class Defined

***GradientProjection*** (network, ods, use_simpy,SO , tol=1e-4 )

<ul><ul>
        : description of the function. 
        <br><br>
        <u> Parameters </u> :
        <ul><ul><br>
                <li>**network** _(object)_ - networkx graph object </li>
                <li>**ods** _(tuple)_ - od demand tuple </li>
                <li>**use_simpy** _(True/False)_ - Optimization tools </li>
                <li>**tol** _(float)_ - converging criteria </li>
        </ul></ul>

## Gradient Projection Algorithm

$min Z = \sum_{a} \int_{0}^{x_a} t_a(\omega)d\omega$ <br>
$\sum_{k} f_k^{rs}= q^{rs}$ <br>
$f_k^{rs} \geq 0$ <br>
$x_{a}= \sum_{\forall (r,s)} \sum_{k \in K_{rs}} f_{k}^{rs}\delta_{k}^{rs}(a) $ <br>
$\delta_{k}^{rs}(a)=1$ if link $a$ is in the route $k$ of pair $(r,s)$ <br>
$K_{rs}$ is the set of paths on pair (r,s) so $k \in K_{rs}$

### Step 0: Initialization

The first is apply the all or nothing assignment based on $t_a=t_a(0)$ ($f_{rs}^1=q^{rs}$), add the path to the path-set.

### Step 1: update
Update $t^n=t_a(x_a^n)$ based on path-link incidence matrix and path flows.

### Step 2: Direction finding

Find the shortest paths from each origin $r$ to each destination $s$ based on $t_a(x_a^n)$; if not on the path set, add path to the path-set.

### Step 3: Move

$f_k^{n+1,rs}=max \{0,f_k^{n,rs}-\frac{\alpha^n}{s_k^{n,rs}}(d_k^{n,rs}-\bar{d}_{rs}^\bar{k} \}$ <br>
where $\bar{d}_{rs}$ is the derivative of the shortest path (travel time) on iteration $k$.
$s_k^n = \sum_{a} \frac{d}{dt}t_a^n$ where a denotes links that are on either k or $\bar{k}_{rs}$ but not on both. 

$f_{\bar{k}}^{n+1,rs}=q_{rs}-\sum_{k \neq \bar{k}}f_k^{n+1,rs}$

In [2]:
class GradientProjection(object):
    def __init__(self, network, ods, use_simpy, SO, tol=1e-4):
        self.network = network
        self.ods = ods
        self.step = 0.6
        self.total_trips = 0
        self.tol = tol
        self.use_simpy = use_simpy
        self.SO = SO
        self.edges = self.getedges()

    def getedges(self):
        edges = []
        for f in self.network.nodes():
            for t in self.network[f]:
                edges.append((f, t))
        return edges

    def solve(self):
        solution = {}

        by_id_links = {}
        for (node_from, node_to) in self.edges:
            l = self.network.get_edge_data(node_from, node_to)['object']
            l.flow = 0
            by_id_links[l.link_id] = l

        od_pairs = []

        for origin in self.ods:
            paths = nx.shortest_path(self.network, origin, weight='time')
            for destination in self.ods[origin]:
                od_pairs.append((origin, destination))
                node_list = paths[destination]
                link_list = []
                for ind in range(len(node_list) - 1):
                    node_from = node_list[ind]
                    node_to = node_list[ind + 1]
                    edge_data = self.network.get_edge_data(node_from, node_to)
                    edge_data['object'].flow += self.ods[origin][destination]
                    # by_id_links[edge_data['object'].link_id].flow += self.ods[origin][destination]
                    if self.SO == True:
                        edge_data['time'] = edge_data['object'].getmarginalcost_l(edge_data['object'].flow)
                        if edge_data['object'].getmarginalcost_l(edge_data['object'].flow) < 0:
                            raise ValueError
                    else:
                        edge_data['time'] = edge_data['object'].get_time()
                    link_list.append(edge_data['object'].link_id)

                self.total_trips += self.ods[origin][destination]

                solution[(origin, destination)] = {tuple(link_list): self.ods[origin][destination]}

        for link_id in by_id_links:
            edge_data = self.network.get_edge_data(by_id_links[link_id].from_node, by_id_links[link_id].to_node)
            if self.SO == True:
                edge_data['time'] = edge_data['object'].getmarginalcost_l(edge_data['object'].flow)
                # if edge_data['object'].getmarginalcost_l(edge_data['object'].flow)<0:
                #     raise ValueError
            else:
                edge_data['time'] = edge_data['object'].get_time()

        objetive_function_history = []
        for i in range(300):
            # random.shuffle(od_pairs)
            for (origin, destination) in od_pairs:
                # for origin in self.ods:
                #     for destination in self.ods[origin]:
                paths = nx.shortest_path(self.network, origin, weight='time')
                node_list = paths[destination]

                link_sequence = []
                for ind in range(len(node_list) - 1):
                    node_from = node_list[ind]
                    node_to = node_list[ind + 1]

                    edge_data = self.network.get_edge_data(node_from, node_to)
                    link = edge_data['object']
                    link_sequence.append(link.link_id)

                link_sequence = tuple(link_sequence)
                # costs, previous = dijkstra_helper.get_shortest_paths(origin)
                # link_sequence = dijkstra_helper.get_link_sequence(origin, destination, previous)
                # link_sequence = tuple(link_sequence)
                if not link_sequence in solution[(origin, destination)]:
                    solution[(origin, destination)][link_sequence] = 0.0

                derivatives = {}
                s_paths = {}
                for path in solution[(origin, destination)]:
                    e_derivative = 0.0
                    s = 0.0

                    for link_id in path:
                        link = by_id_links[link_id]
                        if self.use_simpy == False:
                            e_derivative += link.get_time()
                        else:
                            if self.SO == True:
                                e_derivative += link.getmarginalcost_l(link.flow)
                            else:
                                e_derivative += link.actualcost
                        if link_id not in link_sequence:
                            if self.use_simpy == True:
                                if self.SO == True:
                                    diff = derivative(link.getmarginalcost_l, link.flow)
                                    if diff < 0 or diff > 10000:
                                        print("%.6f" % diff)
                                        diff = 0.0001

                                    s += diff
                                else:
                                    s += derivative(link.getbprcost, link.flow)
                                    a = math.pow(link.flow,
                                                 link.beta - 1) * link.beta * link.alpha * link.length / (
                                            link.free_speed * math.pow(link.capacity, link.beta))
                                    b = derivative(link.getbprcost, link.flow)
                                    if a - b > 0.0001:
                                        print("%.10f" % (a - b))
                            else:
                                s += math.pow(link.flow, link.beta - 1) * link.beta * link.alpha * link.length / (
                                link.free_speed * math.pow(link.capacity, link.beta))  # 1/31/2018 DS

                    derivatives[path] = e_derivative

                    if path != link_sequence:
                        for link_id in link_sequence:
                            if link_id not in path:
                                if self.use_simpy == True:
                                    if self.SO == True:
                                        s += derivative(link.getmarginalcost_l, link.flow)
                                    else:
                                        s += derivative(link.getbprcost, link.flow)
                                        a = math.pow(link.flow,
                                                     link.beta - 1) * link.beta * link.alpha * link.length / (
                                                link.free_speed * math.pow(link.capacity, link.beta))
                                        b = derivative(link.getbprcost, link.flow)
                                        if a - b > 0.0001:
                                            print("%.10f" % (a - b))
                                else:
                                    s += math.pow(link.flow, link.beta - 1) * link.beta * link.alpha * link.length / (
                                        link.free_speed * math.pow(link.capacity, link.beta))
                        s_paths[path] = s
                        # s_paths[path] = 1.0

                total_flow = 0.0
                for path in s_paths:
                    try:
                        previous_flow = solution[(origin, destination)][path]
                    except:
                        previous_flow = 0.0

                    if derivatives[path] < derivatives[link_sequence]:
                        print('-- path', path)
                        print('--shortest', link_sequence)
                        print('--od', origin, destination)
                        print(derivatives[path], derivatives[link_sequence])

                        raise Exception

                    flow = solution[(origin, destination)][path] - self.step * (
                        derivatives[path] - derivatives[link_sequence]) / s_paths[path]

                    flow = max(0.0, flow)
                    total_flow += flow

                    if flow > 0:
                        solution[(origin, destination)][path] = flow
                    else:
                        del solution[origin, destination][path]

                    for link_id in path:
                        by_id_links[link_id].flow += (flow - previous_flow)
                        # if by_id_links[link_id].flow<0:
                        #     raise ValueError
                        edge_data = self.network.get_edge_data(by_id_links[link_id].from_node,
                                                               by_id_links[link_id].to_node)
                        # if by_id_links[link_id].flow != link.flow:
                        #     link.flow =by_id_links[link_id].flow
                        if self.SO == True:
                            edge_data['time'] = by_id_links[link_id].getmarginalcost_l(by_id_links[link_id].flow)
                        else:
                            edge_data['time'] = by_id_links[link_id].get_time()

                try:
                    previous_flow = solution[(origin, destination)][link_sequence]
                except:
                    previous_flow = 0.0

                solution[(origin, destination)][link_sequence] = self.ods[origin][destination] - total_flow
                if self.ods[origin][destination] - total_flow < 0:
                    raise ValueError
                for link_id in link_sequence:
                    by_id_links[link_id].flow += (solution[(origin, destination)][link_sequence] - previous_flow)
                    # if by_id_links[link_id].flow - total_flow < 0:
                    #     raise ValueError
                    edge_data = self.network.get_edge_data(by_id_links[link_id].from_node, by_id_links[link_id].to_node)
                    # if by_id_links[link_id].flow != link.flow:
                    #     link.flow = by_id_links[link_id].flow
                    if self.SO == True:
                        edge_data['time'] = max(0, by_id_links[link_id].getmarginalcost_l(by_id_links[link_id].flow))
                    else:
                        edge_data['time'] = max(0, by_id_links[link_id].get_time())

            objetive_function_history.append(self.convergence_test(solution, by_id_links))
            if objetive_function_history[-1] < self.tol:
                break

        return solution, by_id_links, objetive_function_history

    def get_bpr_objetive_function(self, solution):
        s = 0.0
        for edge in self.edges:
            s += self.network.get_edge_data(edge[0], edge[1])['object'].get_bpr_objective_function()

        return s
    def get_total_traveltime(self, solution):
        s = 0.0
        for edge in self.edges:
            s += self.network.get_edge_data(edge[0], edge[1])['object'].get_total_travel_time_function()

        return s

    def get_objetive_function_SO(self, solution):
        s = 0.0
        for edge in self.edges:
            # print ("obj SO", edge[0], edge[1], self.network.get_edge_data(edge[0], edge[1])['object'].flow, self.network.get_edge_data(edge[0], edge[1])['object'].get_objective_function(), self.network.get_edge_data(edge[0], edge[1])['object'].get_objective_function_SO())
            s += self.network.get_edge_data(edge[0], edge[1])['object'].get_objective_function_SO()

        return s

    def convergence_test(self, solution, by_link_id):

        total_gap = 0

        for origin in self.ods:

            for destination in self.ods[origin]:

                times = {}

                min_t = 10E5
                index_min = None
                for path in solution[origin, destination]:
                    t = 0
                    for link_id in path:
                        link = by_link_id[link_id]
                        t += link.get_time()

                    times[path] = t

                    if t < min_t:
                        min_t = t
                        index_min = path

                if len(solution[origin, destination]) <= 1:
                    continue

                for path in solution[origin, destination]:
                    total_gap += solution[origin, destination][path] * (times[path] - min_t)
        print("UE Objective:", self.get_bpr_objetive_function(solution), "SO Objective:",self.get_total_traveltime(solution))

        return total_gap / (self.total_trips)

## Solution

SiouxFalls Network for UE and SO will be examined to test the GP algorithm.  
You can see the detailed information about Sioux-Falls Network in the Data Folder.


In [3]:
datafolder="./Data/TransportationNetworks/SiouxFalls/"
link_file = datafolder+"SiouxFalls_net.tntp"
trip_file = datafolder+"SiouxFalls_trips.tntp"
node_file =datafolder+"SiouxFalls_node.tntp"


visualization = False
bch = loadProblem.Network(link_file, trip_file, node_file)
# link_fields = {"from": 0, "to": 1, "capacity": 2, "length": 3, "t0": 4, "B": 5, "beta": 6, "V": 7}
# bch.link_fields = link_fields
bch.SO=True
use_simpy = False
if bch.SO==True:
    use_simpy=True
# bch.open_link_file()
bch.open_trip_file()

graph, ods = bch.graph, bch.get_od_dic()

gp = GradientProjection(graph, ods, use_simpy, bch.SO, 1e-3)

solution, by_id_links, objetive_function_history_UE = gp.solve()
e_function_history_UE = gp.solve()

UE Objective: 4619072.39123 SO Objective: 8445685.04787
UE Objective: 4459175.3064 SO Objective: 7801589.10128


UE Objective: 4400679.27003 SO Objective: 7555324.3493
UE Objective: 4367173.18756 SO Objective: 7420676.41633
UE Objective: 4342987.37886 SO Objective: 7352874.3098


UE Objective: 4324246.11353 SO Objective: 7296696.3132
UE Objective: 4310880.31653 SO Objective: 7249129.54315
UE Objective: 4303720.65468 SO Objective: 7224676.33951
UE Objective: 4299868.96731 SO Objective: 7216660.75994
UE Objective: 4297805.72979 SO Objective: 7211529.72166
UE Objective: 4297401.31795 SO Objective: 7208066.18817
UE Objective: 4296981.27813 SO Objective: 7205125.86412


UE Objective: 4296635.27963 SO Objective: 7202464.30667
UE Objective: 4296125.60546 SO Objective: 7200618.32359


UE Objective: 4295668.66157 SO Objective: 7198927.07087
UE Objective: 4295329.95504 SO Objective: 7197301.73279
UE Objective: 4295125.11181 SO Objective: 7195746.57059
UE Objective: 4295053.43834 SO Objective: 7195237.37697
UE Objective: 4294754.86939 SO Objective: 7195102.04363


...

UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289
UE Objective: 4295669.79135 SO Objective: 7194256.05289


UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289


UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289


UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289


UE Objective: 4295669.79136 SO Objective: 7194256.05289
UE Objective: 4295669.79136 SO Objective: 7194256.05289


In [31]:
bch.a()

AttributeError: 'Network' object has no attribute 'a'

In [12]:
def get_OD_path_flow(odpair, solution, by_id_links, verbose=False):
    e_ODpair = odpair
    pathset=[]
    
    for path in solution[e_ODpair]:
        t = 0
        path_link=""
        for link_id in path:
            t += by_id_links[link_id].get_time()
            path_link += str(link_id+1)+","
        pathset.append({"path":path, "volume":solution[e_ODpair][path], "Traveltime":t})
        if verbose==True:
            print("-------------------------------------------------------------------------------------------------------")
            print('vol:%.2f, time:%.2f'%(solution[e_ODpair][path], t), 'path:', path_link)


    return pathset

In [14]:
odpair=('13','17')
paths=get_OD_path_flow(odpair, solution_UE,by_id_links_UE,  verbose=True)

-------------------------------------------------------------------------------------------------------
('vol:285.74, time:45.30', 'path:', '38,36,32,30,')
-------------------------------------------------------------------------------------------------------
('vol:214.26, time:45.30', 'path:', '38,35,6,9,13,25,30,')


In [15]:
paths=get_OD_path_flow(odpair, solution_SO,by_id_links_SO, verbose=True)

-------------------------------------------------------------------------------------------------------
('vol:92.69, time:49.02', 'path:', '38,35,6,10,32,30,')
-------------------------------------------------------------------------------------------------------
('vol:294.60, time:42.62', 'path:', '38,36,32,30,')
-------------------------------------------------------------------------------------------------------
('vol:112.72, time:48.22', 'path:', '38,35,6,9,13,25,30,')


In [16]:
def print_link_stats(graph):
    for f in graph:
        for t in graph[f]:
            link_time = graph[f][t]["object"].get_time()
            link_flow = graph[f][t]["object"].flow
            link_VHT = link_time*link_flow
            link_VMT = link_flow*graph[f][t]["object"].link_length

            print(f,t, link_flow, link_time, link_VHT,link_VMT)

In [12]:
print_link_stats(graph_SO)

('24', '13', 10545.026861262624, 15.041859590450455, 158616.81342464086, 42180.107445050497)
('24', '21', 9783.789379374688, 10.238614869180703, 100172.45141659792, 29351.368138124064)
('24', '23', 7686.022223304908, 3.5739215628753085, 27469.240556608231, 15372.044446609816)
('20', '19', 8712.5602396527593, 9.520122106382463, 82944.637340707122, 34850.240958611037)
('20', '18', 20708.164029769388, 4.367787005972321, 90448.849766770159, 82832.656119077554)
('20', '21', 6359.0562752535443, 8.245128540671908, 52431.236386831799, 38154.337651521266)
('20', '22', 7117.0188495436869, 7.8991514803738205, 56218.409981221397, 35585.094247718436)
('21', '24', 9837.4949940832248, 10.398866455504534, 102298.79670016581, 29512.484982249676)
('21', '20', 6314.8670844542321, 8.183370273818218, 51676.895582035882, 37889.202506725393)
('21', '22', 7937.6923550527117, 3.591920676651756, 28511.561295014406, 15875.384710105423)
('22', '20', 7083.5754736551462, 7.845041107168697, 55570.940776556599, 35417

In [14]:
if visualization ==True:
    #v.NeworkX2Pyplot(graph)
    #v.NetworkX2_Nodes_Shapefile(graph)
    v.NetworkX2_Links_Shapefile(graph, filename="SO_1")
print (gp.get_objetive_function(solution_SO),gp.get_objetive_function_SO(solution_SO))
print('--')

(7194256.0528930053, 21687187.678703856)
--


## History



- 12/9/2017  Gradient Projection for UE was coded by Felipe

- 2/13/2018  Added a code for Visualization and graph export to ArcGIS format

- 5/3/2018   Minor code revision by Daisik

- 5/3/2018   Added System Optimum function